# Set Up

In [1]:
import pandas as pd

In [2]:
index_torgo = pd.read_csv('index_TORGO.txt', sep = '|')
index_torgo_filtered = index_torgo[(~index_torgo['prompt'].str.contains(']'))
                                   & (index_torgo['speaker'] == 'F01')
                                   & (~index_torgo['prompt'].str.contains('/'))]
index_torgo_filtered['url'] =   'https://s3.amazonaws.com/capstone-voice-for-all/TORGO/' \
                                + index_torgo_filtered['speaker'] + '/' \
                                + index_torgo_filtered['session'] + '/' \
                                + index_torgo_filtered['mic'] + '/' \
                                + index_torgo_filtered['prompt_id'] + '.wav'
index_torgo_filtered['filepath'] = '/home/jovyan/work/data/TORGO/' \
                                + index_torgo_filtered['speaker'] + '/' \
                                + index_torgo_filtered['session'] + '/' \
                                + index_torgo_filtered['mic'] + '/' \
                                + index_torgo_filtered['prompt_id'] + '.wav'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [3]:
pd.options.display.max_colwidth = 200
index_torgo_filtered_top10 = index_torgo_filtered[['prompt', 'url', 'filepath']].sample(10)
index_torgo_filtered_top10

,prompt,url,filepath
11,meat,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0012.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0012.wav
43,grow,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0044.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0044.wav
33,feet,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0034.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0034.wav
13,he slowly takes a short walk in the open air each day.,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0014.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0014.wav
144,meat,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0011.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0011.wav
164,The quick brown fox jumps over the lazy dog.,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0031.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0031.wav
232,stubble,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0099.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0099.wav
234,bug,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0101.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0101.wav
27,"but he always answers, Banana oil!",https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0028.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0028.wav
213,hair,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0080.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0080.wav


## Amazon Transcribe

[Info about Amazon Transcribe](https://aws.amazon.com/transcribe/)

In [4]:
from __future__ import print_function
from urllib.request import urlopen
import time
import boto3
import json

In [5]:
def amazon_translate(row):
    jobName = str(row.name)
    jobUri = row.url
    transcribe = boto3.client('transcribe')
    try:
        transcribe.delete_transcription_job(TranscriptionJobName=jobName)
    except:
        pass
    try:
        transcribe.start_transcription_job(
            TranscriptionJobName=jobName,
            Media={'MediaFileUri': jobUri},
            MediaFormat='wav',
            LanguageCode='en-US'
        )
    except:
        pass
    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=jobName)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        print(jobName + ' not ready yet...')
        time.sleep(20)
    result_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
    with urlopen(result_url) as conn:
        transcript_html = conn.read()
        transcript = json.loads(transcript_html)
        transcript_text = transcript['results']['transcripts'][0]['transcript']
        transcribe.delete_transcription_job(TranscriptionJobName=jobName)
        print(jobName + ': ' + transcript_text)
        return transcript_text

In [6]:
index_torgo_filtered_top10['awstranslate'] = index_torgo_filtered_top10.apply(amazon_translate, axis=1)
index_torgo_filtered_top10[['prompt', 'awstranslate']]

11 not ready yet...
11 not ready yet...
11 not ready yet...
11 not ready yet...
11: Hey!
43 not ready yet...
43 not ready yet...
43 not ready yet...
43 not ready yet...
43: Go.
33 not ready yet...
33 not ready yet...
33 not ready yet...
33 not ready yet...
33: 
13 not ready yet...
13 not ready yet...
13 not ready yet...
13 not ready yet...
13 not ready yet...
13: He's slowing kick a slow one in the open in a day.
144 not ready yet...
144 not ready yet...
144 not ready yet...
144 not ready yet...
144: May.
164 not ready yet...
164 not ready yet...
164 not ready yet...
164 not ready yet...
164 not ready yet...
164: The quick brown for guns.
232 not ready yet...
232 not ready yet...
232 not ready yet...
232 not ready yet...
232: Good morning.
234 not ready yet...
234 not ready yet...
234 not ready yet...
234 not ready yet...
234: Well.
27 not ready yet...
27 not ready yet...
27 not ready yet...
27 not ready yet...
27 not ready yet...
27: Book here. Oh, and gloom.
213 not ready yet...
213 

,prompt,awstranslate
11,meat,Hey!
43,grow,Go.
33,feet,
13,he slowly takes a short walk in the open air each day.,He's slowing kick a slow one in the open in a day.
144,meat,May.
164,The quick brown fox jumps over the lazy dog.,The quick brown for guns.
232,stubble,Good morning.
234,bug,Well.
27,"but he always answers, Banana oil!","Book here. Oh, and gloom."
213,hair,Who?


## Google Cloud Speech-to-Text

[Info about Speech-to-Text](https://cloud.google.com/speech-to-text/)

In [7]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

In [8]:
def google_translate(row):
    jobName = str(row.name)
    file_name = row.filepath
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='en-US')

    # Detects speech in the audio file
    response = client.recognize(config, audio)
    try:
        transcript = response.results[0].alternatives[0].transcript
        return transcript
    except:
        return None

In [9]:
index_torgo_filtered_top10

,prompt,url,filepath,awstranslate
11,meat,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0012.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0012.wav,Hey!
43,grow,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0044.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0044.wav,Go.
33,feet,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0034.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0034.wav,
13,he slowly takes a short walk in the open air each day.,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0014.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0014.wav,He's slowing kick a slow one in the open in a day.
144,meat,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0011.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0011.wav,May.
164,The quick brown fox jumps over the lazy dog.,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0031.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0031.wav,The quick brown for guns.
232,stubble,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0099.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0099.wav,Good morning.
234,bug,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0101.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0101.wav,Well.
27,"but he always answers, Banana oil!",https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0028.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0028.wav,"Book here. Oh, and gloom."
213,hair,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0080.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0080.wav,Who?


In [10]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jovyan/work/capstone-c60aec7ead62.json"
client = speech.SpeechClient()

index_torgo_filtered_top10['googletranslate'] = index_torgo_filtered_top10.apply(google_translate, axis=1)
index_torgo_filtered_top10[['prompt', 'googletranslate']]

,prompt,googletranslate
11,meat,None
43,grow,grow
33,feet,None
13,he slowly takes a short walk in the open air each day.,he's slowly a short walk in the open
144,meat,None
164,The quick brown fox jumps over the lazy dog.,the quick brown fox
232,stubble,gummo
234,bug,None
27,"but he always answers, Banana oil!",None
213,hair,hair


In [11]:
index_torgo_filtered_top10[['prompt', 'googletranslate', 'awstranslate']]

,prompt,googletranslate,awstranslate
11,meat,None,Hey!
43,grow,grow,Go.
33,feet,None,
13,he slowly takes a short walk in the open air each day.,he's slowly a short walk in the open,He's slowing kick a slow one in the open in a day.
144,meat,None,May.
164,The quick brown fox jumps over the lazy dog.,the quick brown fox,The quick brown for guns.
232,stubble,gummo,Good morning.
234,bug,None,Well.
27,"but he always answers, Banana oil!",None,"Book here. Oh, and gloom."
213,hair,hair,Who?
